In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
df = pd.read_csv('../gen_data/train2.csv')

y_train_list = pickle.load(open('../gen_data/y_train--features1.ipynb--.pickle','rb'))
x_train_list = pickle.load(open('../gen_data/x_train--features3.ipynb--.pickle','rb'))
x_test_list = pickle.load(open('../gen_data/x_test--features3.ipynb--.pickle','rb'))

In [25]:
for i in range(0,3):
    print(i)

0
1
2


In [181]:
def target_encode(x_train,y_train,x_test,cols,nperms):
    x_train['target'] = y_train

#for test data
    for col in cols:
        agged_sum = x_train.groupby(col).target.sum()
        x_test[col+'_encoded'] = x_test[col].map(agged_sum)
        x_test[col+'_encoded'].fillna(0,inplace=True)
            
#for train data, need to regularise
    for col in cols:
        print('Encoding '+col+':')
        encoding_df = pd.DataFrame()
        for perm_num in range(0,nperms):
            print('\tComputing encoding on permutation '+str(perm_num)+' of '+str(nperms))
            #set random state to perm num for reproducibility
            perm_x_train = x_train.sample(frac=1,random_state=perm_num)
            
            cumsum   = perm_x_train.groupby(col).target.cumsum() - perm_x_train.target
            cumcount = perm_x_train.groupby(col).target.cumcount()
            encoding = cumsum/cumcount
            
            #encoding_df = pd.concat([],axis=1)
            encoding_df['perm_'+str(perm_num)+'_encoding'] = encoding
            
        encoding_df['colmeans'] = encoding_df.mean(axis=1)    
        x_train[col+'_encoded'] = encoding_df.colmeans
        x_train[col+'_encoded'].fillna(0,inplace=True)

    return x_train.drop('target',axis=1) , x_test

In [150]:
x_train = x_train_list[0]
y_train = y_train_list[0]
x_test = x_test_list[0]

In [158]:
x_train_out , x_test_out,enc_df = target_encode(x_train,y_train,x_test,['item_id'],10)

Encoding item_id:
	Computing encoding on permutation 0 of 10
	Computing encoding on permutation 1 of 10
	Computing encoding on permutation 2 of 10
	Computing encoding on permutation 3 of 10
	Computing encoding on permutation 4 of 10
	Computing encoding on permutation 5 of 10
	Computing encoding on permutation 6 of 10
	Computing encoding on permutation 7 of 10
	Computing encoding on permutation 8 of 10
	Computing encoding on permutation 9 of 10


In [152]:
x_test_out.head()[['item_id','item_id_encoded']]

,item_id,item_id_encoded
0,5037,1244.0
1,5320,0.0
2,5233,465.0
3,5232,113.0
4,5268,0.0


In [173]:
x_train_out['target'] = y_train
a = x_train_out[['item_id','item_id_encoded','target']]
out_g = a.groupby('item_id').item_id_encoded.mean()
out_g.head(20)

item_id
0     0.014902
1     0.027294
2     0.011917
3     0.026355
4     0.019736
5     0.020576
6     0.017507
7     0.021025
8     0.026939
9     0.022318
10    0.015396
11    0.019282
13    0.019330
14    0.009379
15    0.021535
16    0.015005
17    0.023381
18    0.025180
20    0.011220
21    0.015649
Name: item_id_encoded, dtype: float32

# is mean really a good idea?

In [174]:
naive_g = a.groupby('item_id').target.mean()
naive_g.head(20)

item_id
0     0.020000
1     0.023810
2     0.019802
3     0.019802
4     0.020000
5     0.020000
6     0.020000
7     0.020000
8     0.019802
9     0.019608
10    0.020000
11    0.020000
13    0.020000
14    0.020000
15    0.020000
16    0.020000
17    0.020000
18    0.019608
20    0.019608
21    0.020000
Name: target, dtype: float32

In [177]:
naive_g.shape

(16561,)

In [179]:
np.corrcoef(naive_g,out_g)

array([[1.        , 0.99952922],
       [0.99952922, 1.        ]])

In [180]:
enc_df

,perm_0_encoding,perm_1_encoding,perm_2_encoding,perm_3_encoding,perm_4_encoding,perm_5_encoding,perm_6_encoding,perm_7_encoding,perm_8_encoding,perm_9_encoding,colmeans
1981796,NaN,0.195122,0.191736,0.162907,0.200000,0.158771,0.164062,0.165605,0.204188,0.150289,0.176965
2386002,NaN,0.024390,0.028351,0.033582,0.033079,0.031339,0.030233,0.045249,0.026923,0.033175,0.031813
1511809,NaN,0.152941,0.273743,0.244936,0.186851,0.232662,0.166667,0.238596,0.208333,0.235043,0.215530
1318036,NaN,0.160976,0.140000,0.148438,0.142384,0.135246,0.167123,0.208791,0.100000,0.101852,0.144979
3348638,NaN,0.047619,0.038674,0.048035,0.043011,0.065217,0.048387,0.045977,0.076923,0.056701,0.052283
2831915,NaN,0.814493,0.684539,0.750000,0.786982,0.716012,0.422535,0.638889,0.759184,0.723473,0.699567
1957489,NaN,0.054348,0.066667,0.051724,0.027027,0.039474,0.081081,0.057971,0.000000,0.050000,0.047588
4770989,NaN,0.039326,0.028796,0.032751,0.034483,0.033505,0.036797,0.000000,0.030405,0.043956,0.031113
4059022,NaN,0.206799,0.261194,0.257042,0.344444,0.242075,0.242347,0.304348,0.200000,0.375000,0.270361
5495464,NaN,0.191263,0.244604,0.200795,0.190174,0.193995,0.214022,0.206816,0.201248,0.205339,0.205362


In [155]:
enc_df

,perm_0_encoding,perm_1_encoding,colmeans
942565,NaN,0.083333,0.083333
4323518,NaN,0.042056,0.042056
2450948,NaN,0.319879,0.319879
200794,NaN,0.030303,0.030303
3037275,NaN,0.059259,0.059259
5697526,NaN,0.254032,0.254032
4426031,NaN,0.153226,0.153226
5022237,NaN,0.051136,0.051136
1317526,NaN,0.323099,0.323099
855577,NaN,0.139430,0.139430


In [157]:
enc_df.sort_values('perm_1_encoding',ascending=False)

,perm_0_encoding,perm_1_encoding,colmeans
5891957,9.823810,20.000000,14.911905
1350279,1.713028,20.000000,10.856514
6156502,15.578947,20.000000,17.789474
6115791,6.500000,20.000000,13.250000
6095530,12.714286,20.000000,16.357143
5202276,1.125189,20.000000,10.562594
6131078,11.482759,20.000000,15.741379
5332635,6.400000,20.000000,13.200000
143599,6.085592,20.000000,13.042796
4507270,17.672728,20.000000,18.836365


# start exp

In [126]:
df = pd.DataFrame({'A':[4,7,6,5,4,4],'B':['red','blue','red','blue','blue','red'],'C':[65,56,76,87,34,33]})
df

,A,B,C
0,4,red,65
1,7,blue,56
2,6,red,76
3,5,blue,87
4,4,blue,34
5,4,red,33


In [127]:
cumsum = df.groupby('B').C.cumsum() - df.C
cumsum

0      0
1      0
2     65
3     56
4    143
5    141
Name: C, dtype: int64

In [128]:
cumcount = df.groupby('B').C.cumcount()
cumcount

0    0
1    0
2    1
3    1
4    2
5    2
dtype: int64

In [129]:
df_encoding = cumsum / cumcount
df['newcol'] = df_encoding
df

,A,B,C,newcol
0,4,red,65,NaN
1,7,blue,56,NaN
2,6,red,76,65.0
3,5,blue,87,56.0
4,4,blue,34,71.5
5,4,red,33,70.5


In [130]:
df_new = df.sample(frac=1)
df_new

,A,B,C,newcol
3,5,blue,87,56.0
5,4,red,33,70.5
4,4,blue,34,71.5
1,7,blue,56,NaN
0,4,red,65,NaN
2,6,red,76,65.0


In [131]:
cumsum = df_new.groupby('B').C.cumsum() - df.C
cumsum

0     33
1    121
2     98
3      0
4     87
5      0
Name: C, dtype: int64

In [132]:
cumcount = df_new.groupby('B').C.cumcount()
cumcount

3    0
5    0
4    1
1    2
0    1
2    2
dtype: int64

In [133]:
df_new_encoding = cumsum / cumcount
df_new['newcol'] = df_new_encoding
df_new.sort_index()

,A,B,C,newcol
0,4,red,65,33.0
1,7,blue,56,60.5
2,6,red,76,49.0
3,5,blue,87,NaN
4,4,blue,34,87.0
5,4,red,33,NaN


In [134]:
df

,A,B,C,newcol
0,4,red,65,NaN
1,7,blue,56,NaN
2,6,red,76,65.0
3,5,blue,87,56.0
4,4,blue,34,71.5
5,4,red,33,70.5


In [135]:
encoding_df = pd.concat([df_new_encoding,df_encoding],axis=1)

encoding_df['colmeans'] = encoding_df.mean(axis=1)
encoding_df


,0,1,colmeans
0,33.0,NaN,33.00
1,60.5,NaN,60.50
2,49.0,65.0,57.00
3,NaN,56.0,56.00
4,87.0,71.5,79.25
5,NaN,70.5,70.50


In [ ]:
df = df.drop('newcol',axis=1)

In [136]:
df['mean_encoding'] = encoding_df.colmeans
df

,A,B,C,mean_encoding
0,4,red,65,33.00
1,7,blue,56,60.50
2,6,red,76,57.00
3,5,blue,87,56.00
4,4,blue,34,79.25
5,4,red,33,70.50


In [138]:
print(np.mean([65,76,33]))
print(np.mean([33,57,70.5]))

58.0
53.5


In [139]:
pd.concat(df.drop('mean_encoding',axis=1),df.mean_encoding)

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

# end exp

In [6]:
x_train_small = x_train[x_train.item_id.isin([31,32])]
x_train_small[['item_id','target']]

KeyError: "['target'] not in index"

In [40]:
cumsum = x_train_small.groupby('item_id').target.cumsum() - x_train_small.target
cumcount = x_train_small.groupby('item_id').target.cumcount()
encoding = cumsum/cumcount

x_train_small['cumsum'] = cumsum
x_train_small['cumcount'] = cumcount
x_train_small['item_id_encoding'] = encoding
x_train_small[['item_id','target','cumsum','cumcount','item_id_encoding']]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#

,item_id,target,cumsum,cumcount,item_id_encoding
0,32,1.0,0.0,0,NaN
896,31,0.0,0.0,0,NaN
7551,32,0.0,1.0,1,1.000000
8447,31,0.0,0.0,1,0.000000
15102,32,0.0,1.0,2,0.500000
15998,31,1.0,0.0,2,0.000000
22653,32,3.0,1.0,3,0.333333
23549,31,1.0,1.0,3,0.333333
30204,32,2.0,4.0,4,1.000000
31100,31,1.0,2.0,4,0.500000


In [24]:
x_train.head()

,date_block_num,shop_id,item_id,item_category_id,sum_item_sales_back_1,sum_shop_sales_back_1,item_cnt_month_back_1,sum_item_cat_sales_back_1,sum_item_cat_shop_sales_back_1,sum_item_sales_back_2,...,item_cnt_month_back_12,sum_item_cat_sales_back_12,sum_item_cat_shop_sales_back_12,month,numdays,mean_prevmonth_item_price,sd_prevmonth_item_price,mean_prevmonth_shop_item_price,sd_above_market,target
0,12,2,32,40.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,jan,31,134.691573,15.602202,NaN,0.000000,1.0
1,12,2,33,37.0,42.0,1255.0,1.0,9906.0,55.0,15.0,...,1.0,5982.0,52.0,jan,31,186.571429,16.449056,199.0,0.755580,1.0
2,12,2,99,37.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,jan,31,124.167083,25.367019,NaN,0.000000,1.0
3,12,2,482,73.0,88.0,1255.0,1.0,412.0,3.0,85.0,...,1.0,664.0,7.0,jan,31,2866.022727,435.543311,3300.0,0.996404,2.0
4,12,2,485,73.0,111.0,1255.0,1.0,412.0,3.0,0.0,...,0.0,0.0,0.0,jan,31,250.203390,51.145216,300.0,0.973632,1.0


In [19]:
x_train_out , x_test_out = target_encode(x_train,y_train,x_test,['item_id'])
x_test_out.sort_values(['item_id']).head(100)

,date_block_num,shop_id,item_id,item_category_id,sum_item_sales_back_1,sum_shop_sales_back_1,item_cnt_month_back_1,sum_item_cat_sales_back_1,sum_item_cat_shop_sales_back_1,sum_item_sales_back_2,...,item_cnt_month_back_12,sum_item_cat_sales_back_12,sum_item_cat_shop_sales_back_12,month,numdays,mean_prevmonth_item_price,sd_prevmonth_item_price,mean_prevmonth_shop_item_price,sd_above_market,item_id_encoded
130087,33,47,30,40.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,oct,31,119.000000,17.320508,NaN,0.000000,300.0
33187,33,10,30,40.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,oct,31,119.000000,17.320508,NaN,0.000000,300.0
22987,33,2,30,40.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,oct,31,119.000000,17.320508,NaN,0.000000,300.0
160687,33,56,30,40.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,oct,31,119.000000,17.320508,NaN,0.000000,300.0
28087,33,7,30,40.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,oct,31,119.000000,17.320508,NaN,0.000000,300.0
17887,33,3,30,40.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,oct,31,119.000000,17.320508,NaN,0.000000,300.0
7687,33,4,30,40.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,oct,31,119.000000,17.320508,NaN,0.000000,300.0
38287,33,12,30,40.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,oct,31,119.000000,17.320508,NaN,0.000000,300.0
114787,33,49,30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,oct,31,119.000000,17.320508,NaN,0.000000,300.0
181087,33,38,30,40.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,oct,31,119.000000,17.320508,NaN,0.000000,300.0


In [18]:
x_train['target'] = y_train
x_train.groupby('item_id').target.sum()

item_id
0          1.0
1          6.0
2          2.0
3          2.0
4          1.0
5          1.0
6          1.0
7          1.0
8          2.0
9          1.0
10         1.0
11         1.0
13         1.0
14         1.0
15         1.0
16         1.0
17         1.0
18         1.0
20         1.0
21         1.0
22         1.0
23         1.0
24         1.0
25         1.0
26         1.0
27         6.0
28         1.0
29         1.0
30       300.0
31       324.0
         ...  
22132      5.0
22133     30.0
22134      1.0
22135     21.0
22136      6.0
22139    188.0
22140    228.0
22141     53.0
22142     10.0
22143    732.0
22144    116.0
22145    306.0
22146     27.0
22147      4.0
22149      2.0
22150     14.0
22152      3.0
22153      9.0
22154     18.0
22155     16.0
22156      1.0
22159     15.0
22160      1.0
22162    645.0
22163     49.0
22164    462.0
22165      1.0
22166    248.0
22167    900.0
22169      1.0
Name: target, Length: 16561, dtype: float32

In [2]:
def target_encode(df,target,cols,fillna=''):
    df['target'] = target
    for col in cols:
        print('Encoding variable: '+col)
        agged_targ = df.groupby(col).target.sum()
        df[col] = df[col].map(agged_targ)
        if fillna=='mean':
            df[col].fillna(df(col).mean(),inplace=True)
        elif fillna=='median':
            df[col].fillna(df(col).median(),inplace=True)
    return df.drop('target',axis=1)